In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Setup initial data frame
df = pd.read_csv('survey_results.csv')
cols = ['timestamp', 'age', 'pronouns', 'nationality', 'usage', 'first_page', 'for_you', 'following', 'favorites', 'friends', 'discover', 'search', 
        'like', 'comment', 'save', 'follow', 'share', 'not_interested', 'refresh', 'interesting', 'search_for_you', 'search_scroll', 'search_again', 
        'search_other', 'discover_for_you', 'discover_stay', 'discover_other', 'count_videos_account', 'politics', 'sports', 'eating_habits', 
        'memes', 'conspiracies', 'pets', 'dance', 'entertainment', 'food', 'games', 'other', 'political_topics', 'conspiracy_topics', 
        'eating_habits_topics']
df.columns = cols
len(df)

139

In [3]:
# Clean Data

# Remove participants that do not recognize themselves with the She/Her pronouns
df = df[df['pronouns'] == 'She/her']

# Remove participants that do not use TikTok
df = df[df['usage'] != 'Never']

# Remove non valid age groups (NaN)
df['age'] = df['age'].astype('int64')
df = df[pd.to_numeric(df['age'], errors='coerce').notnull()]

# Separate age into groups (Youth [0-19], Young Adults [20-29], Adults [30-64], Seniors [65+])
age_limits = [0,19,29,64,150]
age_groups = ['Youth','Young Adult','Adult','Senior']
df['age_group'] = pd.cut(df['age'], bins=age_limits, labels=age_groups, right=False)

# Remove age groups that are under-represented (<15%)
counts = df['age_group'].value_counts(normalize=True)
df = df[df['age_group'].isin(counts[counts.gt(0.15)].index)]

len(df)

75

In [4]:
# Participants ages sorted by percentage
df_age = df.groupby(['age'], observed=False)
df_age.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

,age,percentage
0,19,26.67
1,20,29.33
2,21,24.00
3,22,10.67
4,23,2.67
5,24,2.67
6,26,2.67
7,27,1.33


In [5]:
# Participants average age
print("Average age of participants is " + df['age'].mean().astype('int64').astype('str') + " years old")

Average age of participants is 20 years old


In [6]:
# Participants age groups sorted by percentage
df_age_group = df.groupby(['age_group'], observed=False)
df_age_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,age_group,percentage
0,Young Adult,100.0
1,Youth,0.0
2,Adult,0.0
3,Senior,0.0


In [7]:
# Participants pronouns/gender sorted by percentage
df_pronouns_group = df.groupby(['pronouns'], observed=False)
df_pronouns_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,pronouns,percentage
0,She/her,100.0


In [8]:
# Initial TikTok use sorted by percentage
df_usage_group = df.groupby(['usage'], observed=False)
df_usage_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,usage,percentage
0,Multiple times a day,60.00
1,Once everyday,29.33
2,Once in a while,10.67


In [9]:
# First page that participants choose to verify sorted by percentage
df_first_page_group = df.groupby(['first_page'], observed=False)
df_first_page_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,first_page,percentage
0,For You,90.67
1,Discover,2.67
2,Following,2.67
3,Friends,2.67
4,Search,1.33


In [10]:
# Percentage of content deemed interesting by participants in the For You page
df_interesting_group = df.groupby(['interesting'], observed=False)
df_interesting_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,interesting,percentage
0,Often,70.67
1,Always,20.00
2,Sometimes,9.33


In [11]:
# Percentage of numbers of videos watched when checking an account by participants
df_count_videos_account_group = df.groupby(['count_videos_account'], observed=False)
df_count_videos_account_group.size().transform(lambda x: round((x*100)/sum(x),2)).sort_values(ascending=False).reset_index(name='percentage')

,count_videos_account,percentage
0,1-5 videos,77.33
1,Upto 10 vidoes,20.00
2,Never go into accounts,2.67


In [12]:
# Percentage of respondents based on how much they verify a certain page
df_pages = pd.DataFrame()

df_for_you = df.groupby(['for_you'], observed=False)
df_pages[['for_you', 'for_you_prob']] = df_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_following = df.groupby(['following'], observed=False)
df_pages[['following', 'following_prob']] = df_following.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_favorites = df.groupby(['favorites'], observed=False)
df_pages[['favorites', 'favorites_prob']] = df_favorites.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_friends = df.groupby(['friends'], observed=False)
df_pages[['friends', 'friends_prob']] = df_friends.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover = df.groupby(['discover'], observed=False)
df_pages[['discover', 'discover_prob']] = df_discover.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search = df.groupby(['search'], observed=False)
df_pages[['search', 'search_prob']] = df_search.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_pages

,for_you,for_you_prob,following,following_prob,favorites,favorites_prob,friends,friends_prob,discover,discover_prob,search,search_prob
0,Always,78.67,Always,6.67,Always,1.33,Always,2.67,Always,2.67,Always,4.00
1,Never,2.67,Never,21.33,Never,33.33,Never,29.33,Never,54.67,Never,1.33
2,Often,16.00,Often,13.33,Often,14.67,Often,8.00,Often,5.33,Often,38.67
3,Rarely,2.67,Rarely,40.00,Rarely,32.00,Rarely,41.33,Rarely,24.00,Rarely,12.00


In [13]:
# Probability setting of staying, or leaving a page based on participants answers
df_pages = pd.DataFrame()

# ['Never', 'Rarely', 'Sometimes'] = 'Leaving Probability'
df[['for_you', 'following', 'favorites', 'friends', 
    'discover', 'search']] = df[['for_you', 'following', 'favorites', 'friends', 
                                'discover', 'search']].replace(['Never', 'Rarely', 'Sometimes'], 'Leaving Probability')

# ['Often', 'Always'] = 'Staying Probability'
df[['for_you', 'following', 'favorites', 'friends', 
    'discover', 'search']] = df[['for_you', 'following', 'favorites', 'friends', 
                                 'discover', 'search']].replace(['Often', 'Always'], 'Staying Probability')

# Group pages by probability of staying, or leaving a page
df_for_you = df.groupby(['for_you'], observed=False)
df_pages[['for_you', 'for_you_prob']] = df_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_following = df.groupby(['following'], observed=False)
df_pages[['following', 'following_prob']] = df_following.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_favorites = df.groupby(['favorites'], observed=False)
df_pages[['favorites', 'favorites_prob']] = df_favorites.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_friends = df.groupby(['friends'], observed=False)
df_pages[['friends', 'friends_prob']] = df_friends.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover = df.groupby(['discover'], observed=False)
df_pages[['discover', 'discover_prob']] = df_discover.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search = df.groupby(['search'], observed=False)
df_pages[['search', 'search_prob']] = df_search.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_pages

,for_you,for_you_prob,following,following_prob,favorites,favorites_prob,friends,friends_prob,discover,discover_prob,search,search_prob
0,Leaving Probability,5.33,Leaving Probability,80.0,Leaving Probability,84.0,Leaving Probability,89.33,Leaving Probability,92.0,Leaving Probability,57.33
1,Staying Probability,94.67,Staying Probability,20.0,Staying Probability,16.0,Staying Probability,10.67,Staying Probability,8.0,Staying Probability,42.67


In [14]:
# Percentage of respondents based on how much they use a certain feature
df_features = pd.DataFrame()

df_like = df.groupby(['like'], observed=False)
df_features[['like', 'like_p']] = df_like.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_comment = df.groupby(['comment'], observed=False)
df_features[['comment', 'comment_p']] = df_comment.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_save = df.groupby(['save'], observed=False)
df_features[['save', 'save_p']] = df_save.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_follow = df.groupby(['follow'], observed=False)
df_features[['follow', 'follow_p']] = df_follow.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_share = df.groupby(['share'], observed=False)
df_features[['share', 'share_p']] = df_share.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_not_interested = df.groupby(['not_interested'], observed=False)
df_features[['not_interested', 'not_interested_p']] = df_not_interested.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_refresh = df.groupby(['refresh'], observed=False)
df_features[['refresh', 'refresh_p']] = df_refresh.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_features

,like,like_p,comment,comment_p,save,save_p,follow,follow_p,share,share_p,not_interested,not_interested_p,refresh,refresh_p
0,Always,18.67,Always,10.67,Always,17.33,Always,2.67,Always,17.33,Always,5.33,Always,7.94
1,Never,8.00,Never,36.00,Never,5.33,Never,12.00,Never,8.00,Never,16.00,Never,12.70
2,Often,38.67,Often,4.00,Often,22.67,Often,13.33,Often,42.67,Often,10.67,Often,28.57
3,Rarely,16.00,Rarely,28.00,Rarely,16.00,Rarely,40.00,Rarely,14.67,Rarely,41.33,Rarely,26.98
4,Sometimes,18.67,Sometimes,21.33,Sometimes,38.67,Sometimes,32.00,Sometimes,17.33,Sometimes,26.67,Sometimes,23.81


In [15]:
# Probability setting of using, or not using a feature based on participants usage
df_features = pd.DataFrame()

# ['Never', 'Rarely', 'Sometimes'] = 'Not Using Probability'
df[['like', 'comment', 'save', 'follow', 'share', 
    'not_interested', 'refresh']] = df[['like', 'comment', 'save', 'follow', 'share', 
                                        'not_interested', 'refresh']].replace(['Never', 'Rarely', 'Sometimes'], 'Not Using Probability')

# ['Often', 'Always'] = 'Using Probability'
df[['like', 'comment', 'save', 'follow', 'share', 
    'not_interested', 'refresh']] = df[['like', 'comment', 'save', 'follow', 'share', 
                                        'not_interested', 'refresh']].replace(['Often', 'Always'], 'Using Probability')

# Group pages by probability of using, or not using a feature
df_like = df.groupby(['like'], observed=False)
df_features[['like', 'like_p']] = df_like.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_comment = df.groupby(['comment'], observed=False)
df_features[['comment', 'comment_p']] = df_comment.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_save = df.groupby(['save'], observed=False)
df_features[['save', 'save_p']] = df_save.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_follow = df.groupby(['follow'], observed=False)
df_features[['follow', 'follow_p']] = df_follow.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_share = df.groupby(['share'], observed=False)
df_features[['share', 'share_p']] = df_share.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_not_interested = df.groupby(['not_interested'], observed=False)
df_features[['not_interested', 'not_interested_p']] = df_not_interested.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_refresh = df.groupby(['refresh'], observed=False)
df_features[['refresh', 'refresh_p']] = df_refresh.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_features

,like,like_p,comment,comment_p,save,save_p,follow,follow_p,share,share_p,not_interested,not_interested_p,refresh,refresh_p
0,Not Using Probability,42.67,Not Using Probability,85.33,Not Using Probability,60.0,Not Using Probability,84.0,Not Using Probability,40.0,Not Using Probability,84.0,Not Using Probability,63.49
1,Using Probability,57.33,Using Probability,14.67,Using Probability,40.0,Using Probability,16.0,Using Probability,60.0,Using Probability,16.0,Using Probability,36.51


In [16]:
# Percentage of respondents based on how much they do a certain action after executing a search
df_search = pd.DataFrame()

df_search_for_you = df.groupby(['search_for_you'], observed=False)
df_search[['search_for_you', 'search_for_you_p']] = df_search_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_scroll = df.groupby(['search_scroll'], observed=False)
df_search[['search_scroll', 'search_scroll_p']] = df_search_scroll.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_again = df.groupby(['search_again'], observed=False)
df_search[['search_again', 'search_again_p']] = df_search_again.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_other = df.groupby(['search_other'], observed=False)
df_search[['search_other', 'search_other_p']] = df_search_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search

,search_for_you,search_for_you_p,search_scroll,search_scroll_p,search_again,search_again_p,search_other,search_other_p
0,Always,25.33,Always,8.00,Always,2.67,Never,61.33
1,Never,1.33,Never,4.00,Never,8.00,Often,1.33
2,Often,56.00,Often,44.00,Often,18.67,Rarely,20.00
3,Rarely,5.33,Rarely,9.33,Rarely,25.33,Sometimes,17.33
4,Sometimes,12.00,Sometimes,34.67,Sometimes,45.33,NaN,NaN


In [17]:
# Probability setting of doing, or not doing an action when inside the search page based on participants usage
df_search = pd.DataFrame()

# ['Never', 'Rarely', 'Sometimes'] = 'Not Doing Probability'
df[['search_for_you', 'search_scroll', 
    'search_again', 'search_other']] = df[['search_for_you', 'search_scroll', 
                                           'search_again', 'search_other']].replace(['Never', 'Rarely', 'Sometimes'], 'Not Doing Probability')

# ['Often', 'Always'] = 'Doing Probability'
df[['search_for_you' ,'search_scroll', 
    'search_again', 'search_other']] = df[['search_for_you', 'search_scroll', 
                                           'search_again', 'search_other']].replace(['Often', 'Always'], 'Doing Probability')

# Group pages by probability of doing, or not doing an action
df_search_for_you = df.groupby(['search_for_you'], observed=False)
df_search[['search_for_you', 'search_for_you_p']] = df_search_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_scroll = df.groupby(['search_scroll'], observed=False)
df_search[['search_scroll', 'search_scroll_p']] = df_search_scroll.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_again = df.groupby(['search_again'], observed=False)
df_search[['search_again', 'search_again_p']] = df_search_again.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search_other = df.groupby(['search_other'], observed=False)
df_search[['search_other', 'search_other_p']] = df_search_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_search

,search_for_you,search_for_you_p,search_scroll,search_scroll_p,search_again,search_again_p,search_other,search_other_p
0,Doing Probability,81.33,Doing Probability,52.0,Doing Probability,21.33,Doing Probability,1.33
1,Not Doing Probability,18.67,Not Doing Probability,48.0,Not Doing Probability,78.67,Not Doing Probability,98.67


In [18]:
# Percentage of respondents based on how much they do a certain action after executing a search
df_discover = pd.DataFrame()

df_discover_for_you = df.groupby(['discover_for_you'], observed=False)
df_discover[['discover_for_you', 'discover_for_you_prob']] = df_discover_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover_stay = df.groupby(['discover_stay'], observed=False)
df_discover[['discover_stay', 'discover_stay_prob']] = df_discover_stay.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover_other = df.groupby(['discover_other'], observed=False)
df_discover[['discover_other', 'discover_other_prob']] = df_discover_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover

,discover_for_you,discover_for_you_prob,discover_stay,discover_stay_prob,discover_other,discover_other_prob
0,Always,28.00,Always,1.33,Never,66.67
1,Never,17.33,Never,48.00,Often,5.33
2,Often,41.33,Often,9.33,Rarely,14.67
3,Rarely,4.00,Rarely,21.33,Sometimes,13.33
4,Sometimes,9.33,Sometimes,20.00,NaN,NaN


In [19]:
# Probability setting of not doing, or not doing an action when inside the discover page based on participants usage
df_discover = pd.DataFrame()

# ['Never', 'Rarely', 'Sometimes'] = 'Not Doing Probability'
df[['discover_for_you', 'discover_stay', 
    'discover_other']] = df[['discover_for_you', 'discover_stay', 'discover_other']].replace(['Never', 'Rarely', 'Sometimes'], 'Not Doing Probability')

# ['Often', 'Always'] = 'Doing Probability'
df[['discover_for_you' ,'discover_stay', 
    'discover_other']] = df[['search_for_you', 'discover_stay', 'discover_other']].replace(['Often', 'Always'], 'Doing Probability')

# Group pages by probability of doing, or not doing an action
df_discover_for_you = df.groupby(['discover_for_you'], observed=False)
df_discover[['discover_for_you', 'discover_for_you_prob']] = df_discover_for_you.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover_stay = df.groupby(['discover_stay'], observed=False)
df_discover[['discover_stay', 'discover_stay_prob']] = df_discover_stay.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover_other = df.groupby(['discover_other'], observed=False)
df_discover[['discover_other', 'discover_other_prob']] = df_discover_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_discover

,discover_for_you,discover_for_you_prob,discover_stay,discover_stay_prob,discover_other,discover_other_prob
0,Doing Probability,81.33,Doing Probability,10.67,Doing Probability,5.33
1,Not Doing Probability,18.67,Not Doing Probability,89.33,Not Doing Probability,94.67


In [20]:
# Percentage of respondents based on how much they watch a certain video given their topic
df_topics = pd.DataFrame()

df_topics_politics = df.groupby(['politics'], observed=False)
df_topics[['politics', 'politics_prob']] = df_topics_politics.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_sports = df.groupby(['sports'], observed=False)
df_topics[['sports', 'sports_prob']] = df_topics_sports.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_eating_habits = df.groupby(['eating_habits'], observed=False)
df_topics[['eating_habits', 'eating_habits_prob']] = df_topics_eating_habits.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_memes = df.groupby(['memes'], observed=False)
df_topics[['memes', 'memes_prob']] = df_topics_memes.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_conspiracies = df.groupby(['conspiracies'], observed=False)
df_topics[['conspiracies', 'conspiracies_prob']] = df_topics_conspiracies.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_pets = df.groupby(['pets'], observed=False)
df_topics[['pets', 'pets_prob']] = df_topics_pets.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_dance = df.groupby(['dance'], observed=False)
df_topics[['dance', 'dance_prob']] = df_topics_dance.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_entertainment = df.groupby(['entertainment'], observed=False)
df_topics[['entertainment', 'entertainment_prob']] = df_topics_entertainment.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_food = df.groupby(['food'], observed=False)
df_topics[['food', 'food_prob']] = df_topics_food.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_games = df.groupby(['games'], observed=False)
df_topics[['games', 'games_prob']] = df_topics_games.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_other = df.groupby(['other'], observed=False)
df_topics[['other', 'other_prob']] = df_topics_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics

,politics,politics_prob,sports,sports_prob,eating_habits,eating_habits_prob,memes,memes_prob,conspiracies,conspiracies_prob,pets,pets_prob,dance,dance_prob,entertainment,entertainment_prob,food,food_prob,games,games_prob,other,other_prob
0,Always,1.33,Always,1.33,Always,1.33,Always,18.67,Always,1.33,Always,12.00,Always,6.67,Always,22.67,Always,25.33,Always,4.00,Always,18.67
1,Never,12.00,Never,21.33,Never,10.67,Never,4.00,Never,34.67,Never,12.00,Never,25.33,Never,5.33,Never,4.00,Never,34.67,Never,18.67
2,Often,25.33,Often,9.33,Often,22.67,Often,37.33,Often,10.67,Often,37.33,Often,17.33,Often,44.00,Often,37.33,Often,13.33,Often,26.67
3,Rarely,29.33,Rarely,36.00,Rarely,26.67,Rarely,9.33,Rarely,37.33,Rarely,8.00,Rarely,25.33,Rarely,2.67,Rarely,8.00,Rarely,32.00,Rarely,9.33
4,Sometimes,32.00,Sometimes,32.00,Sometimes,38.67,Sometimes,30.67,Sometimes,16.00,Sometimes,30.67,Sometimes,25.33,Sometimes,25.33,Sometimes,25.33,Sometimes,16.00,Sometimes,26.67


In [21]:
# Probability setting for watching, or not watching a video of a certain topic based on participants answers
df_topics = pd.DataFrame()

# ['Never', 'Rarely', 'Sometimes'] = 'Do Not Watch'
df[['politics', 'sports', 'eating_habits', 'memes', 'conspiracies', 'pets', 'dance', 'entertainment', 'food', 'games', 
    'other']] = df[['politics', 'sports', 'eating_habits', 'memes', 'conspiracies', 'pets', 'dance', 'entertainment', 'food', 'games', 
                    'other']].replace(['Never', 'Rarely', 'Sometimes'], 'Do Not Watch')

# ['Often', 'Always'] = 'Watch'
df[['politics', 'sports', 'eating_habits', 'memes', 'conspiracies', 'pets', 'dance', 'entertainment', 'food', 'games', 
    'other']] = df[['politics', 'sports', 'eating_habits', 'memes', 'conspiracies', 'pets', 'dance', 'entertainment', 'food', 'games', 
                    'other']].replace(['Often', 'Always'], 'Watch')

# Group topics by probability of watching, or not watching a video
df_topics_politics = df.groupby(['politics'], observed=False)
df_topics[['politics', 'politics_prob']] = df_topics_politics.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_sports = df.groupby(['sports'], observed=False)
df_topics[['sports', 'sports_prob']] = df_topics_sports.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_eating_habits = df.groupby(['eating_habits'], observed=False)
df_topics[['eating_habits', 'eating_habits_prob']] = df_topics_eating_habits.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_memes = df.groupby(['memes'], observed=False)
df_topics[['memes', 'memes_prob']] = df_topics_memes.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_conspiracies = df.groupby(['conspiracies'], observed=False)
df_topics[['conspiracies', 'conspiracies_prob']] = df_topics_conspiracies.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_pets = df.groupby(['pets'], observed=False)
df_topics[['pets', 'pets_prob']] = df_topics_pets.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_dance = df.groupby(['dance'], observed=False)
df_topics[['dance', 'dance_prob']] = df_topics_dance.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_entertainment = df.groupby(['entertainment'], observed=False)
df_topics[['entertainment', 'entertainment_prob']] = df_topics_entertainment.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_food = df.groupby(['food'], observed=False)
df_topics[['food', 'food_prob']] = df_topics_food.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_games = df.groupby(['games'], observed=False)
df_topics[['games', 'games_prob']] = df_topics_games.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

df_topics_other = df.groupby(['other'], observed=False)
df_topics[['other', 'other_prob']] = df_topics_other.size().transform(lambda x: round((x*100)/sum(x),2)).reset_index(name='percentage')

In [22]:
# Probability setting for watching, or not watching a video of a certain topic based on participants answers (first half)
df_topics[['politics', 'politics_prob', 'sports', 'sports_prob', 'eating_habits', 'eating_habits_prob', 'memes', 'memes_prob',
    'conspiracies', 'conspiracies_prob']]

,politics,politics_prob,sports,sports_prob,eating_habits,eating_habits_prob,memes,memes_prob,conspiracies,conspiracies_prob
0,Do Not Watch,73.33,Do Not Watch,89.33,Do Not Watch,76.0,Do Not Watch,44.0,Do Not Watch,88.0
1,Watch,26.67,Watch,10.67,Watch,24.0,Watch,56.0,Watch,12.0


In [23]:
# Probability setting for watching, or not watching a video of a certain topic based on participants answers (second half)
df_topics[['pets', 'pets_prob', 'dance', 'dance_prob', 'entertainment', 'entertainment_prob', 'food', 'food_prob', 'games', 'games_prob', 'other', 
          'other_prob']]

,pets,pets_prob,dance,dance_prob,entertainment,entertainment_prob,food,food_prob,games,games_prob,other,other_prob
0,Do Not Watch,50.67,Do Not Watch,76.0,Do Not Watch,33.33,Do Not Watch,37.33,Do Not Watch,82.67,Do Not Watch,54.67
1,Watch,49.33,Watch,24.0,Watch,66.67,Watch,62.67,Watch,17.33,Watch,45.33
